# Simple AI Workflow with Reflection using OpenRouter

In this demo, we build a simple AI workflow that demonstrates the **reflection pattern** — a technique where multiple LLM calls work together to iteratively improve output quality.

Our workflow consists of three steps, each handled by an LLM with a distinct persona:

1. **Report Writer** (think journalist) — Generates an initial report based on a user-provided topic
2. **Critic** (think editor) — Provides detailed critique and improvement suggestions
3. **Reviser** (think senior reporter) — Takes the original report and critique to produce a polished final version

This pattern illustrates a fundamental concept in AI systems: **a single LLM call often isn't enough**. By orchestrating multiple calls with different roles, we can achieve significantly better results than any single call would produce.

We use the OpenAI SDK directly (with OpenRouter as our provider) to keep the implementation transparent — no frameworks, just straightforward API calls you can follow step by step.

# Installs and Imports

In [21]:
%%capture
!pip install openai python-dotenv

`%%capture` hides the pip install output to keep the notebook clean.

In [22]:
# OpenAI SDK: provides a clean interface for calling LLMs
# We'll use it with OpenRouter, which is OpenAI API-compatible
from openai import OpenAI

# Standard library
import os

**OpenRouter** is a unified API that provides access to various LLMs through a single interface. It offers a generous free tier and affordable token usage for minimal cost, making it ideal for learning and experimentation.

If you already pay for other LLM providers or prefer to use a different service, you are welcome to adapt the code accordingly — the OpenAI SDK works with any OpenAI-compatible API.

# Setup your API Key

## Step 1 — Get an OpenRouter API key

For this demo we will use an LLM via OpenRouter, which requires an API key.

1. Go to https://openrouter.ai

2. Sign in (or create an account if you don't have one)

3. Once logged in, navigate to https://openrouter.ai/settings/keys

4. Click **Create Key**

5. Give the key a name, e.g. `colab-reflection-workflow`

6. Copy the key immediately (you won't be able to see it again)

**Important: Treat this key like a password. Do not share it, paste it into notebooks, or commit it to GitHub.**

## Step 2 — Store the API key

### If running in Google Colab:

1. On the left sidebar, click 🔑 **Secrets**

2. Add a new secret:
   - Name: `OPENROUTER_API_KEY`
   - Value: your actual API key

3. Toggle the switch to give notebook access (you should see a checkmark)

### If running locally:

1. Create a `.env` file in the project root:
   ```
   touch .env
   ```

2. Add the following (replace with your own key):
   ```
   OPENROUTER_API_KEY=your_openrouter_key_here
   ```

**Important: Never paste API keys directly into code cells.**

## Load the API Key

### In Colab:

Uncomment and run the cell below if you're using Google Colab.

In [23]:
# # Uncomment the lines below if running in Google Colab
# from google.colab import userdata
# os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
# print("API key loaded from Colab Secrets")

### Locally:

Run the cell below if you're running locally with a `.env` file.

In [24]:
# Load API key from .env file if running locally
from dotenv import load_dotenv
load_dotenv()
print("Environment loaded from .env file")

Environment loaded from .env file


In [25]:
# Sanity check: verify the API key is available
api_key = os.getenv("OPENROUTER_API_KEY")
assert api_key is not None, "OPENROUTER_API_KEY not found. Please check your setup."
print("OPENROUTER_API_KEY present:", bool(api_key))

OPENROUTER_API_KEY present: True


# Initialize the LLM Client

We use the OpenAI SDK to communicate with OpenRouter. Since OpenRouter is OpenAI API-compatible, we simply point the client to OpenRouter's base URL.

In this demo we use `gpt-4o-mini`, a lightweight and cost-efficient model that works well for text generation tasks. You may change the model by selecting one available on the [OpenRouter model list](https://openrouter.ai/models).

In [26]:
# Initialize the OpenAI client pointing to OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

# Model to use for all three personas
MODEL = "openai/gpt-4o-mini"

print(f"Client initialized. Using model: {MODEL}")

Client initialized. Using model: openai/gpt-4o-mini


# Step 1: The Report Writer

Our workflow begins with the **Report Writer** — an LLM prompted to act as a journalist who writes an initial report on a given topic.

## How LLM Personas Work

To give an LLM a specific persona, we use the **system prompt**. This is a special message that sets the context and behavior for the model before it sees the user's input. The system prompt is not part of the conversation — it's instructions to the model about *how* to respond.

The OpenAI SDK uses a simple message format:
```python
messages = [
    {"role": "system", "content": "You are a..."},  # Sets the persona
    {"role": "user", "content": "..."}               # The actual request
]
```

For our Report Writer:
- **System prompt**: Instructs the LLM to behave as a journalist writing informative reports
- **User message**: The topic the user wants a report on, wrapped with additional instructions

Note that the user message doesn't have to be the raw user input verbatim. We can add context or instructions around it to guide the LLM's response — for example, wrapping the topic with "Write a concise, well-structured report on the following topic: {topic}".

The LLM then generates a response as if it were that journalist, producing our initial draft.

In [27]:
# System prompt that defines the Report Writer persona
WRITER_SYSTEM_PROMPT = """You are a journalist who writes clear, informative reports.

When given a topic, write a short report (2-3 paragraphs) that:
- Provides a balanced overview of the topic
- Includes relevant context and background
- Is written in a professional, accessible style

Write only the report itself, without any preamble or meta-commentary."""

In [28]:
def generate_initial_report(topic: str) -> str:
    """
    Generate an initial report on the given topic using the Report Writer persona.
    
    Args:
        topic: The subject the user wants a report on
        
    Returns:
        The generated report as a string
    """
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": WRITER_SYSTEM_PROMPT},
            {"role": "user", "content": f"Write a concise, well-structured report on the following topic: {topic}"}
        ],
        temperature=0.7  # Higher temperature for creative, varied writing
    )
    
    # Extract the text content from the response
    return response.choices[0].message.content

The function above:
1. Takes a `topic` string from the user
2. Wraps it with instructions ("Write a concise, well-structured report on the following topic:")
3. Sends it to the LLM along with our journalist system prompt
4. Returns the generated report text

We set `temperature=0.7` to allow some creativity in the writing. Lower values (closer to 0) make the output more deterministic, while higher values (up to 1) make it more varied.

Let's test the Report Writer with a sample topic:

### Pedagogical note:
For pedagogical purposes, let's inspect what's in the response object

In [29]:
response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": WRITER_SYSTEM_PROMPT},
            {"role": "user", "content": f"Write a concise, well-structured report on the following topic: The impact of artificial intelligence on healthcare"}
        ],
        temperature=0.7  # Some creativity for writing
    )

In [30]:
response.model_dump()

{'id': 'gen-1768498630-wJVacGcHBLCiYXhLW2RA',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'Artificial intelligence (AI) is transforming the healthcare sector by enhancing diagnostic accuracy, personalizing treatment plans, and streamlining administrative processes. AI technologies, such as machine learning and natural language processing, are increasingly being integrated into various aspects of healthcare, from predictive analytics that identify patient risks to AI-powered imaging tools that assist radiologists in detecting abnormalities. A notable example is the use of AI algorithms to analyze medical images, which has shown to improve the speed and reliability of diagnoses for conditions such as cancer and cardiovascular diseases.\n\nDespite the promising benefits, the integration of AI in healthcare also raises significant challenges. Concerns about data privacy and security are paramount, as AI systems often require access t

Back to how we would use our `generate_initial_report` function:

In [31]:
# Test the Report Writer with a sample topic
topic = "The impact of artificial intelligence on healthcare"

initial_report = generate_initial_report(topic)
initial_report

"Artificial intelligence (AI) is transforming the healthcare sector by enhancing diagnostics, personalizing treatment, and improving operational efficiency. AI technologies, such as machine learning and natural language processing, enable the analysis of vast amounts of medical data, leading to quicker and more accurate diagnoses. For instance, AI algorithms can analyze medical imaging with precision, often identifying conditions like cancer at earlier stages than human radiologists. This capability is particularly crucial in areas where timely intervention can significantly improve patient outcomes.\n\nMoreover, AI is instrumental in personalizing patient treatment plans. By utilizing patient data, including genetic information and historical health records, AI can suggest tailored therapies that are more effective for individual patients. This shift towards personalized medicine not only enhances treatment efficacy but also minimizes adverse drug reactions. However, the integration o

# Step 2: The Critic

Now that we have an initial report, we pass it to the **Critic** — an LLM prompted to act as an editor who reviews the work and provides constructive feedback.

The Critic doesn't rewrite the report. Instead, it identifies:
- Areas that need clarification or more detail
- Structural or logical issues
- Missing perspectives or context
- Ways to improve clarity and impact

This separation of concerns (writing vs. critiquing) is key to the reflection pattern. By having a dedicated critic, we get focused, actionable feedback rather than a muddled attempt to both evaluate and fix at once.

In [32]:
# System prompt that defines the Critic persona
CRITIC_SYSTEM_PROMPT = """You are an editor who provides constructive feedback on reports.

When given a report, provide a detailed critique that:
- Identifies areas that need more clarity or detail
- Points out any structural or logical issues
- Suggests missing perspectives or context that would strengthen the piece
- Offers specific, actionable suggestions for improvement

Be constructive and specific. Focus on how to make the report better, not just what's wrong with it.
Do not rewrite the report — only provide feedback."""

In [33]:
def generate_critique(report: str) -> str:
    """
    Generate a critique of the given report using the Critic persona.
    
    Args:
        report: The report to critique
        
    Returns:
        The critique as a string
    """
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": CRITIC_SYSTEM_PROMPT},
            {"role": "user", "content": f"Please critique the following report:\n\n{report}"}
        ],
        temperature=0.5  # Lower temperature for focused, systematic analysis
    )
    
    return response.choices[0].message.content

Notice how the output of Step 1 (the initial report) becomes the input to Step 2. This is the essence of chaining LLM calls — each step builds on the previous one.

We use a lower `temperature=0.5` here because we want the critique to be thoughtful and focused rather than wildly creative.

Let's see what the Critic thinks of our initial report:

In [34]:
# Generate critique of the initial report
critique = generate_critique(initial_report)
critique

"Thank you for sharing your report on the impact of artificial intelligence in the healthcare sector. Below are some constructive critiques to enhance clarity, structure, and depth:\n\n1. **Clarity and Detail**:\n   - **Definition of AI**: While you mention AI and its technologies, consider providing a brief definition or explanation of what artificial intelligence encompasses, especially for readers who may not be familiar with the term.\n   - **Examples of AI Technologies**: You mention machine learning and natural language processing but do not elaborate on how these technologies specifically apply in healthcare. Adding a few sentences to describe their roles could enhance understanding.\n   - **Specific Examples**: While you mention AI's ability to analyze medical imaging, including specific examples of AI tools or platforms currently in use (e.g., IBM Watson Health, Google DeepMind) would provide concrete evidence of AI's impact.\n\n2. **Structural and Logical Issues**:\n   - **Fl

In [35]:
len(critique)

3441

# Step 3: The Reviser

The **Reviser** — prompted as a senior reporter — receives both the original report and the critique, then produces a polished final version.

This is where the reflection pattern pays off. The Reviser doesn't start from scratch; it has:
- The initial draft (what was written)
- The critique (what could be improved)

By combining these inputs, the Reviser can make targeted improvements while preserving what already worked well.

In [36]:
# System prompt that defines the Reviser persona
REVISER_SYSTEM_PROMPT = """You are a senior reporter who produces polished, final versions of reports.

You will receive an original report and editorial feedback. Your task is to revise the report by:
- Addressing the specific points raised in the critique
- Improving clarity, structure, and flow
- Maintaining the original voice and intent
- Ensuring the final version is publication-ready

Write only the revised report, without any preamble or explanation of your changes."""

In [37]:
def generate_revised_report(report: str, critique: str) -> str:
    """
    Generate a revised report using the Reviser persona.
    
    Args:
        report: The original report to revise
        critique: The editorial feedback to address
        
    Returns:
        The revised report as a string
    """
    # Combine both inputs into a structured user message
    user_message = f"""Please revise the following report based on the editorial feedback provided.

ORIGINAL REPORT:
{report}

EDITORIAL FEEDBACK:
{critique}"""
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": REVISER_SYSTEM_PROMPT},
            {"role": "user", "content": user_message}
        ],
        temperature=0.6  # Moderate: follow critique closely but write naturally
    )
    
    return response.choices[0].message.content

Notice how this function takes **two inputs** — the original report and the critique. We structure the user message clearly with labeled sections so the LLM can easily distinguish between them.

Let's generate our final, polished report:

In [38]:
# Generate the final revised report
final_report = generate_revised_report(initial_report, critique)
final_report

"Artificial intelligence (AI) is revolutionizing the healthcare sector by enhancing diagnostics, personalizing treatment, and improving operational efficiency. At its core, AI encompasses the simulation of human intelligence processes by machines, particularly computer systems, and includes technologies such as machine learning, natural language processing, and robotics. These technologies facilitate the analysis of vast amounts of medical data, leading to quicker and more accurate diagnoses.\n\nFor example, machine learning algorithms can analyze medical imaging with remarkable precision, identifying conditions like cancer at earlier stages than human radiologists. Tools such as IBM Watson Health and Google DeepMind exemplify the application of AI in healthcare, demonstrating how these systems can improve diagnostic accuracy and speed. This capability is crucial in areas where timely intervention can significantly enhance patient outcomes.\n\nIn addition to diagnostics, AI plays a vit

In [39]:
len(final_report)

3725

# Comparing the Results

Let's compare the initial report with the final revised version to see how the reflection pattern improved the output:

In [40]:
from IPython.display import Markdown

comparison = f"""
## Initial Report

{initial_report}

---

## Critique

{critique}

---

## Final Report (After Reflection)

{final_report}
"""

Markdown(comparison)


## Initial Report

Artificial intelligence (AI) is transforming the healthcare sector by enhancing diagnostics, personalizing treatment, and improving operational efficiency. AI technologies, such as machine learning and natural language processing, enable the analysis of vast amounts of medical data, leading to quicker and more accurate diagnoses. For instance, AI algorithms can analyze medical imaging with precision, often identifying conditions like cancer at earlier stages than human radiologists. This capability is particularly crucial in areas where timely intervention can significantly improve patient outcomes.

Moreover, AI is instrumental in personalizing patient treatment plans. By utilizing patient data, including genetic information and historical health records, AI can suggest tailored therapies that are more effective for individual patients. This shift towards personalized medicine not only enhances treatment efficacy but also minimizes adverse drug reactions. However, the integration of AI in healthcare also raises ethical and regulatory concerns, including patient privacy, data security, and the potential for bias in AI algorithms. As healthcare organizations increasingly adopt these technologies, addressing these challenges will be essential to ensure that AI's benefits are realized while maintaining patient trust and safety.

---

## Critique

Thank you for sharing your report on the impact of artificial intelligence in the healthcare sector. Below are some constructive critiques to enhance clarity, structure, and depth:

1. **Clarity and Detail**:
   - **Definition of AI**: While you mention AI and its technologies, consider providing a brief definition or explanation of what artificial intelligence encompasses, especially for readers who may not be familiar with the term.
   - **Examples of AI Technologies**: You mention machine learning and natural language processing but do not elaborate on how these technologies specifically apply in healthcare. Adding a few sentences to describe their roles could enhance understanding.
   - **Specific Examples**: While you mention AI's ability to analyze medical imaging, including specific examples of AI tools or platforms currently in use (e.g., IBM Watson Health, Google DeepMind) would provide concrete evidence of AI's impact.

2. **Structural and Logical Issues**:
   - **Flow of Ideas**: The report jumps from diagnostics to personalized treatment without a clear transition. Consider adding a transitional sentence that connects these two sections more smoothly to improve the overall flow.
   - **Ethical and Regulatory Concerns**: The mention of ethical and regulatory issues feels somewhat abrupt. It would benefit from a brief introduction or context about why these concerns are arising alongside AI adoption. Perhaps a sentence summarizing the rapid pace of AI integration into healthcare could set the stage for these challenges.

3. **Missing Perspectives or Context**:
   - **Patient Perspective**: Including a section on how patients perceive AI in their treatment could add depth. Are patients generally receptive to AI-driven diagnostics and treatment plans? Are there concerns or misconceptions that need addressing?
   - **Healthcare Provider Perspective**: Discussing how healthcare professionals view AI's integration into their workflows could provide a balanced perspective. Are there fears of job displacement, or do they see it as a valuable tool?
   - **Global Context**: Consider mentioning how AI adoption in healthcare may vary across different regions or countries. This would provide a broader context and highlight disparities in technology access and healthcare quality.

4. **Actionable Suggestions for Improvement**:
   - **Incorporate Data and Statistics**: Including relevant statistics or studies that quantify AI's impact on healthcare outcomes would strengthen your arguments. For example, citing research that shows improved diagnostic accuracy rates with AI compared to traditional methods would provide empirical support.
   - **Addressing Bias**: When discussing bias in AI algorithms, it would be helpful to elaborate on how bias can be introduced and the potential consequences for patient care. This could lead to a discussion on how to mitigate these biases, which would be a proactive approach to the issue.
   - **Conclusion**: The report lacks a concluding section that summarizes the main points and offers a forward-looking statement. Consider adding a conclusion that encapsulates the benefits and challenges of AI in healthcare, along with a call to action for stakeholders to address the highlighted concerns.

By addressing these areas, your report can become more comprehensive, engaging, and informative for readers interested in the intersection of AI and healthcare.

---

## Final Report (After Reflection)

Artificial intelligence (AI) is revolutionizing the healthcare sector by enhancing diagnostics, personalizing treatment, and improving operational efficiency. At its core, AI encompasses the simulation of human intelligence processes by machines, particularly computer systems, and includes technologies such as machine learning, natural language processing, and robotics. These technologies facilitate the analysis of vast amounts of medical data, leading to quicker and more accurate diagnoses.

For example, machine learning algorithms can analyze medical imaging with remarkable precision, identifying conditions like cancer at earlier stages than human radiologists. Tools such as IBM Watson Health and Google DeepMind exemplify the application of AI in healthcare, demonstrating how these systems can improve diagnostic accuracy and speed. This capability is crucial in areas where timely intervention can significantly enhance patient outcomes.

In addition to diagnostics, AI plays a vital role in personalizing patient treatment plans. By leveraging patient data, including genetic information and historical health records, AI can suggest tailored therapies that are more effective for individual patients. This shift towards personalized medicine not only boosts treatment efficacy but also minimizes adverse drug reactions. However, the integration of AI in healthcare raises important ethical and regulatory concerns, such as patient privacy, data security, and the potential for bias in AI algorithms. As healthcare organizations increasingly adopt these technologies, it is essential to address these challenges to ensure that AI's benefits are realized while maintaining patient trust and safety.

Transitioning from diagnosis to treatment personalization, it is important to consider the perspectives of both patients and healthcare providers. Patients may have varying degrees of receptiveness to AI-driven diagnostics and treatment plans. Some may appreciate the precision and personalization that AI offers, while others might harbor concerns or misconceptions about its role in their care. Healthcare professionals, too, may view AI's integration into their workflows with mixed feelings—some may fear job displacement, while others see it as a valuable tool that enhances their capabilities.

Furthermore, the adoption of AI in healthcare is not uniform across the globe. Variations in technology access and healthcare quality can create disparities in how AI is implemented and its effectiveness in different regions or countries. Addressing these global disparities is crucial as we move forward.

To strengthen the arguments presented, incorporating relevant statistics and studies that quantify AI's impact on healthcare outcomes would be beneficial. For instance, research showing improved diagnostic accuracy rates with AI compared to traditional methods would provide empirical support for the claims made. Additionally, when discussing bias in AI algorithms, it is vital to elaborate on how bias can be introduced and the potential consequences for patient care. A proactive approach to mitigating these biases will be essential as AI continues to evolve.

In conclusion, while AI presents transformative potential for enhancing healthcare through improved diagnostics and personalized treatment, it also introduces ethical and regulatory challenges that must be addressed. Stakeholders in the healthcare sector must work collaboratively to navigate these complexities, ensuring that the benefits of AI are maximized while safeguarding patient trust and safety. As we look to the future, a balanced approach will be key to harnessing AI's capabilities for the betterment of healthcare worldwide.
